# Prompt Tuning with SmolLM: A Beginner's Guide

This notebook demonstrates how to set up and use prompt tuning with a SmolLM model using the PEFT library. Prompt tuning is a parameter-efficient method that adds trainable "soft prompts" while keeping the base model frozen.

## What You'll Learn
- How to load a SmolLM model
- Configure prompt tuning with PEFT
- Train soft prompts for sentiment classification
- Compare performance with and without prompt tuning

## 1. Install Dependencies

First, let's install the required packages:

In [1]:
!pip install transformers peft torch datasets accelerate

## 2. Import Libraries

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import PromptTuningConfig, TaskType, get_peft_model
from datasets import Dataset
import numpy as np

## 3. Load SmolLM Model and Tokenizer

We'll use SmolLM2-135M, a small but capable language model:

In [ ]:
# Load the base model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")

## 4. Test Base Model (Before Prompt Tuning)

Let's see how the model performs on sentiment classification without any training:

In [ ]:
def test_model(model, tokenizer, text, max_length=50):
    """Test the model with a given input text"""
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test with sentiment classification prompt
test_prompt = "Classify the sentiment: 'This movie is amazing!' Sentiment:"
print("Base model response:")
print(test_model(model, tokenizer, test_prompt))

## 5. Configure Prompt Tuning

Now let's set up prompt tuning configuration:

### Parameter Breakdown:

- **`task_type=TaskType.CAUSAL_LM`**: Specifies causal language modeling (next token prediction)
- **`num_virtual_tokens=16`**: Number of trainable soft prompt vectors (typically 8-32)
- **`prompt_tuning_init="TEXT"`**: Initialize from text embeddings (vs "RANDOM")
- **`prompt_tuning_init_text`**: Text that gets converted to initial soft prompt embeddings
- **`tokenizer_name_or_path`**: Tokenizer for encoding the initialization text

The init text gets tokenized and embedded to create starting values for the 16 trainable vectors.

In [ ]:
# Configure prompt tuning
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=16,  # Number of soft prompt tokens
    prompt_tuning_init="TEXT",  # Initialize from text
    prompt_tuning_init_text="Classify the sentiment as positive, negative, or neutral:",
    tokenizer_name_or_path=model_name,
)

print("Prompt tuning configuration:")
print(f"Virtual tokens: {peft_config.num_virtual_tokens}")
print(f"Initialization: {peft_config.prompt_tuning_init}")
print(f"Init text: {peft_config.prompt_tuning_init_text}")

## 6. Create Prompt-Tunable Model

Apply the prompt tuning configuration to our model:

In [ ]:
# Create the prompt-tunable model
model = get_peft_model(model, peft_config)

# Print model info
model.print_trainable_parameters()

print("\nPrompt tuning model created!")
print(f"Only {model.num_parameters(only_trainable=True):,} parameters will be trained")

## 7. Prepare Training Data

Create a simple sentiment classification dataset:

In [ ]:
# Create training data
train_data = [
    {"text": "I love this movie! It's fantastic.", "label": "positive"},
    {"text": "This film is terrible and boring.", "label": "negative"},
    {"text": "The movie was okay, nothing special.", "label": "neutral"},
    {"text": "Amazing acting and great story!", "label": "positive"},
    {"text": "Worst movie I've ever seen.", "label": "negative"},
    {"text": "It's an average film.", "label": "neutral"},
    {"text": "Brilliant cinematography and direction!", "label": "positive"},
    {"text": "Complete waste of time.", "label": "negative"},
    {"text": "The movie is decent.", "label": "neutral"},
    {"text": "Absolutely loved every minute!", "label": "positive"},
]

def format_prompt(example):
    """Format the training examples"""
    return f"Text: {example['text']} Sentiment: {example['label']}"

# Format the data
formatted_data = [format_prompt(ex) for ex in train_data]

print("Training examples:")
for i, example in enumerate(formatted_data[:3]):
    print(f"{i+1}. {example}")

## 8. Tokenize Data

In [ ]:
def tokenize_function(examples):
    """Tokenize the training data"""
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

# Create dataset
dataset = Dataset.from_dict({"text": formatted_data})
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Add labels (for language modeling, labels = input_ids)
def add_labels(example):
    example["labels"] = example["input_ids"].copy()
    return example

tokenized_dataset = tokenized_dataset.map(add_labels)

print(f"Dataset created with {len(tokenized_dataset)} examples")

## 9. Set Up Training

Configure the training arguments and trainer:

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./prompt_tuning_results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    learning_rate=1e-3,  # Higher learning rate for prompt tuning
    logging_steps=5,
    save_strategy="no",
    remove_unused_columns=False,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

print("Trainer configured successfully!")

## 10. Train the Model

Now let's train the soft prompts:

In [ ]:
print("Starting prompt tuning training...")
trainer.train()
print("Training completed!")

## 11. Test the Trained Model

Let's see how the model performs after prompt tuning:

In [ ]:
# Test the trained model
test_examples = [
    "Text: This movie is incredible! Sentiment:",
    "Text: I hated this film. Sentiment:",
    "Text: The movie was fine. Sentiment:",
]

print("Testing prompt-tuned model:")
print("=" * 50)

for i, example in enumerate(test_examples):
    response = test_model(model, tokenizer, example, max_length=len(example.split()) + 5)
    print(f"Test {i+1}:")
    print(f"Input: {example}")
    print(f"Output: {response}")
    print("-" * 30)

## 12. Save the Prompt-Tuned Model

Save only the prompt parameters (very small file!):

In [ ]:
# Save the prompt tuning adapter
model.save_pretrained("./smol_prompt_tuned")

print("Prompt tuning adapter saved!")
print("Only the soft prompt parameters are saved, not the entire model.")

# Check file size
import os
adapter_size = os.path.getsize("./smol_prompt_tuned/adapter_model.bin")
print(f"Adapter file size: {adapter_size / 1024:.2f} KB")

## 13. Load and Use Saved Model

Demonstrate how to load the prompt-tuned model:

In [ ]:
from peft import PeftModel

# Load base model again
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load the prompt tuning adapter
loaded_model = PeftModel.from_pretrained(base_model, "./smol_prompt_tuned")

print("Model loaded successfully!")

# Test the loaded model
test_text = "Text: This is an amazing experience! Sentiment:"
result = test_model(loaded_model, tokenizer, test_text)
print(f"\nTest with loaded model:")
print(f"Input: {test_text}")
print(f"Output: {result}")

## 14. Summary

### What We Accomplished:

1. **Loaded SmolLM2-135M** - A small but capable language model
2. **Configured Prompt Tuning** - Set up 16 virtual tokens initialized from text
3. **Trained Soft Prompts** - Only trained ~8K parameters instead of 135M
4. **Achieved Task Adaptation** - Model learned sentiment classification
5. **Saved Efficiently** - Adapter file is only a few KB

### Key Benefits of Prompt Tuning:

- **Parameter Efficient**: Only 0.006% of model parameters trained
- **Fast Training**: Quick convergence with small dataset
- **Easy Deployment**: Tiny adapter files for different tasks
- **Task Switching**: Can quickly switch between different prompt-tuned tasks

### Next Steps:

- Try different initialization strategies (random vs text)
- Experiment with different numbers of virtual tokens
- Test on more complex tasks
- Compare with LoRA and full fine-tuning approaches